# Compare two GCHP datasets

## Overview of this Notebook

* Import dependencies
* Define functions: utility, regridding, and plotting
* Define data directories and global variables
* Compare restart files
* Store areas for normalization
* Compare SpeciesConc diagnostic collection
* Create PDFs for subset of species
* Other diagnostic collections

## Import dependencies

In [1]:
import os
import numpy as np
import xarray as xr
import cubedsphere as cs
import xesmf as xe
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import ticker
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
from cartopy import crs
from cartopy.mpl.geoaxes import GeoAxes

%matplotlib inline
import warnings; warnings.filterwarnings("ignore")

# Also define colormap. Colormap file source: https://bitbucket.org/gcst/gcpy
rgb_WhGrYlRd = np.genfromtxt('/n/home08/elundgren/GC/python/WhGrYlRd.txt',delimiter=' ')
WhGrYlRd = ListedColormap(rgb_WhGrYlRd/255.0)

## Define general utility functions

In [2]:
def get_gchp_filepath(outputdir, collection, day, time):
    filepath = os.path.join(outputdir, 'GCHP.{}.{}_{}z.nc4'.format(collection,day,time))
    return filepath

In [3]:
def check_paths( refpath, devpath):
    if not os.path.exists(refpath):
        print('ERROR! Path 1 does not exist: {}'.format(refpath))
    else:
        print('Path 1 exists: {}'.format(refpath))
    if not os.path.exists(devpath):
        print('ERROR! Path 2 does not exist: {}'.format(devpath))
    else:
        print('Path 2 exists: {}'.format(devpath))

In [4]:
def compare_varnames(refdata, refstr, devdata, devstr):
    
    # Find common variables in collection by generating lists and list overlap
    refvars = [k for k in refdata.data_vars.keys()]
    devvars= [k for k in devdata.data_vars.keys()]
    commonvars = sorted(list(set(refvars).intersection(set(devvars))))
    refonly = [v for v in refvars if v not in devvars]
    devonly = [v for v in devvars if v not in refvars]
    dimmismatch = [v for v in commonvars if refdata[v].ndim != devdata[v].ndim]
    commonvars2D = [v for v in commonvars if refdata[v].ndim == 3]
    commonvars3D = [v for v in commonvars if devdata[v].ndim == 4]
    
    # Print information on common and mismatching variables, as well as dimensions
    print('{} common variables ({} are 2-dim and {} are 3-dim)'.format(len(commonvars), len(commonvars2D), len(commonvars3D)))
    if len(refonly) > 0:
        print('{} variables in {} only (skip)'.format(len(refnly)),refstr)
        print('   Variable names: {}'.format(refonly))
    if len(devonly) > 0:
        print('{} variables in {} only (skip)'.format(len(devonly)),devstr)
        print('   Variable names: {}'.format(devonly))
    if len(dimmismatch) > 0:
        print('{} common variables have different dimensions'.format(len(dimmismatch)))
        print('   Variable names: {}'.format(dimmismatch))
        
    return [commonvars, commonvars2D, commonvars3D]

In [5]:
def get_stats(refdata, refstr, devdata, devstr, varname):
    refvar = refdata[varname]
    devvar = devdata[varname]
    units = refdata[varname].units
    print('Data units: {}'.format(units))
    print('Array sizes:')
    print('    {}:  {}'.format(refstr,refvar.shape))
    print('    {}:  {}'.format(devstr,devvar.shape))
    print('Global stats:')
    print('  Mean:')
    print('    {}:  {}'.format(refstr,np.round(refvar.values.mean(),20)))
    print('    {}:  {}'.format(devstr,np.round(devvar.values.mean(),20)))
    print('  Min:')
    print('    {}:  {}'.format(refstr,np.round(refvar.values.min(),20)))
    print('    {}:  {}'.format(devstr,np.round(devvar.values.min(),20)))
    print('  Max:')
    print('    {}:  {}'.format(refstr,np.round(refvar.values.max(),20)))
    print('    {}:  {}'.format(devstr,np.round(devvar.values.max(),20)))
    print('  Sum:')
    print('    {}:  {}'.format(refstr,np.round(refvar.values.sum(),20)))
    print('    {}:  {}'.format(devstr,np.round(devvar.values.sum(),20)))

In [6]:
def get_collection_data(refdir, devdir, collection, day, time):
    reffile = get_gchp_filepath(refdir, collection, day, time)
    devfile = get_gchp_filepath(devdir, collection, day, time)
    check_paths(reffile, devfile)
    refdata = xr.open_dataset(reffile)
    devdata = xr.open_dataset(devfile)
    return [refdata, devdata]

## Define functions for regridding

These functions utilize the xESMF package and the cubedsphere package developed by Jaiwei Zhuang, a graduate student at Harvard University. More specifically I use the cubedsphere package CSGrid class and the xESMF package Regridder class. The latter serves as a wrapper for the ESMF/ESMPy packages. These tools enable simple conservative regridding between lat/lon and cubed sphere horizontal grids as specifically defined by the NASA Global Modeling and Assimilation Office (GMAO). See http://xesmf.readthedocs.io/en/latest/ and https://github.com/JiaweiZhuang/cubedsphere for more information. 

In [7]:
def make_grid_LL(llres):
    [dlat,dlon] = list(map(float, llres.split('x')))
    lon_b = np.linspace(-180 - dlon/2, 180 - dlon/2, int(360/dlon) + 1, endpoint=True)
    lat_b = np.linspace(-90 - dlat/2, 90 + dlat/2, int(180/dlat) + 2, endpoint=True).clip(-90,90)
    lat = (lat_b[1:] + lat_b[:-1]) / 2
    lon = (lon_b[1:] + lon_b[:-1]) / 2
    llgrid = {'lat': lat, 
              'lon': lon, 
              'lat_b': lat_b, 
              'lon_b': lon_b}
    return llgrid

In [8]:
def make_grid_CS(csres):
    csgrid = cs.csgrid_GMAO(csres)
    csgrid_list = [None]*6
    for i in range(6):
        csgrid_list[i] = {'lat': csgrid['lat'][i], 
                          'lon': csgrid['lon'][i],
                          'lat_b': csgrid['lat_b'][i], 
                          'lon_b': csgrid['lon_b'][i]}
    return [csgrid, csgrid_list]

In [9]:
def make_regridder_C2L( csres_in, llres_out, weightsdir='.', reuse_weights=False ):
    csgrid, csgrid_list = make_grid_CS(csres_in)
    llgrid = make_grid_LL(llres_out)
    regridder_list = []
    for i in range(6):
        weightsfile = os.path.join(weightsdir, 'conservative_c{}_{}_{}.nc'.format(str(csres_in), llres_out, str(i)))
        regridder = xe.Regridder(csgrid_list[i], llgrid, method='conservative', filename=weightsfile, reuse_weights=reuse_weights)
        regridder_list.append(regridder)
    return regridder_list

In [10]:
def make_regridder_L2L( llres_in, llres_out, weightsdir='.', reuse_weights=False ):
    llgrid_in = make_grid_LL(llres_in)
    llgrid_out = make_grid_LL(llres_out)
    weightsfile = os.path.join(weightsdir,'conservative_{}_{}.nc'.format(llres_in, llres_out))
    regridder = xe.Regridder(llgrid_in, llgrid_out, method='conservative', filename=weightsfile, reuse_weights=reuse_weights)
    return regridder

## Define functions for plotting

### Function to plot and compare single vertical level of GCHP

Notes: This function currently does not regrid GCHP; therefore the two GCHP files must be at the same cubed sphere resolution. You can use this function to plot interactively or to generate a multi-page pdf of plots. It takes a list of variable names to plot for a single collection only. You can plot for any level and any time slice in the file. By default the colorbars for the plots will have the same range, but you can turn this feature off. Also by default the colorbar of the fractional difference between the model outputs will be limited to +/-2, but you can change this as well via the passed parameters.

In [11]:
def compare_single_level(refdata, refstr, devdata, devstr, varlist=None, weightsdir='.', ilev=0, 
                         itime=0, savepdf=False, pdfname='gchp_vs_gchp_map.pdf', match_cbar=True, 
                         full_ratio_range=False, normalize_by_area=False, area_ref=None, 
                         area_dev=None, check_units=True, flip_vert=False):
    
    # If no varlist is passed, plot all (surface only for 3D)
    if varlist == None:
        [varlist, commonvars2D, commonvars3D] = compare_varnames(refdata, devdata)
        print('Plotting all common variables (surface only if 3D)')
    n_var = len(varlist)

    # Get cubed sphere grids and regridder 
    # for now, do not regrid for gchp vs gchp. Assume same grid.
    csres_ref = refdata['lon'].size
    [csgrid_ref, csgrid_list_ref] = make_grid_CS(csres_ref)
    csres_dev = devdata['lon'].size
    [csgrid_dev, csgrid_list_dev] = make_grid_CS(csres_dev)

    # Create pdf (if saving)
    if savepdf:
        print('\nCreating {} for {} variables'.format(pdfname,n_var))
        pdf = PdfPages(pdfname)

    # Loop over variables
    for ivar in range(n_var):
        if savepdf: print('{} '.format(ivar), end='')
        varname = varlist[ivar]
        
        # Do some checks: dimensions and units
        varndim_ref = refdata[varname].ndim
        varndim_dev = devdata[varname].ndim
        if check_units: 
            assert varndim_ref == varndim_dev, 'Dimensions do not agree for {}!'.format(varname)
        units_ref = refdata[varname].units
        units_dev = devdata[varname].units
        if check_units: 
            assert units_ref == units_dev, 'Units do not match for {}!'.format(varname)
            
        # if normalizing by area, adjust units to be per m2, and adjust title string
        units = units_ref
        varndim = varndim_ref
        subtitle_extra = ''
                    
        # Slice the data
        if varndim == 4: 
            if flip_vert: 
                ds_ref = refdata[varname].isel(time=itime,lev=71-ilev)
                ds_dev = devdata[varname].isel(time=itime,lev=71-ilev)
            else: 
                ds_ref = refdata[varname].isel(time=itime,lev=ilev)
                ds_dev = devdata[varname].isel(time=itime,lev=ilev)
        elif varndim == 3: 
            refds = refdata[varname].isel(time=itime)
            devds = devdata[varname].isel(time=itime)
            
        # if normalizing by area, transform on the native grid and adjust units and subtitle string
        exclude_list = ['WetLossConvFrac','Prod_','Loss_']
        if normalize_by_area and not any(s in varname for s in exclude_list):
            ds_ref.values = ds_ref.values / area_ref
            ds_dev.values = ds_dev.values / area_dev
            units = '{} m-2'.format(units)
            subtitle_extra = ', Normalized by Area'
            
        # Regrid the slices (skip for gchp vs gchp for now)
        csdata_ref = ds_ref.data.reshape(6,csres_ref,csres_ref)
        csdata_dev = ds_dev.data.reshape(6,csres_dev,csres_dev)
        vmin_ref = csdata_ref.min()
        vmin_dev = csdata_dev.min()
        vmin_cmn = np.min([vmin_ref, vmin_dev])
        vmax_ref = csdata_ref.max()
        vmax_dev = csdata_dev.max()
        vmax_cmn = np.max([vmax_ref, vmax_dev])
        if match_cbar: [vmin, vmax] = [vmin_cmn, vmax_cmn]
        
        # Create 2x2 figure
        figs, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=[12,9], 
                                                      subplot_kw={'projection': crs.PlateCarree()})
        # Give the figure a title
        offset = 0.96
        fontsize=25
        if varndim == 4:
            if ilev == 0: levstr = 'Surface'
            elif ilev == 22: levstr = '500 hPa'
            else: levstr = 'Level ' +  str(ilev-1)
            figs.suptitle('{}, {}'.format(varname,levstr), fontsize=fontsize, y=offset)
        elif varndim == 3: 
            figs.suptitle('{}'.format(varname), fontsize=fontsize, y=offset)
        else:
            print('varndim is 2 for {}! Must be 3 or 4.'.format(varname))
            
        # Subplot (0,0): Ref
        ax0.coastlines()
        if not match_cbar: [vmin, vmax] = [vmin_ref, vmax_ref]        
        masked_csdata = np.ma.masked_where(np.abs(csgrid_ref['lon'] - 180) < 2, csdata_ref)
        for i in range(6):
            plot0 = ax0.pcolormesh(csgrid_ref['lon_b'][i,:,:], csgrid_ref['lat_b'][i,:,:], masked_csdata[i,:,:], 
                                   cmap=WhGrYlRd,vmin=vmin, vmax=vmax)
        ax0.set_title('{} (Ref){}\nC{}'.format(refstr,subtitle_extra,str(csres_ref)))
        cb = plt.colorbar(plot0, ax=ax0, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.1 or (vmax-vmin) > 100:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)     
        
        # Subplot (0,1): Dev
        ax1.coastlines()
        if not match_cbar: [vmin, vmax] = [vmin_dev, vmax_dev]        
        masked_csdata = np.ma.masked_where(np.abs(csgrid_dev['lon'] - 180) < 2, csdata_dev)
        for i in range(6):
            plot1 = ax1.pcolormesh(csgrid_dev['lon_b'][i,:,:], csgrid_dev['lat_b'][i,:,:], 
                                   masked_csdata[i,:,:], cmap=WhGrYlRd,vmin=vmin, vmax=vmax)
        ax1.set_title('{} (Dev){}\nC{}'.format(devstr,subtitle_extra,str(csres_dev)))
        cb = plt.colorbar(plot1, ax=ax1, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.1 or (vmax-vmin) > 100:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)  
            
        # Subplot (1,0): Difference
        gc_absdiff = csdata_dev - csdata_ref
        diffabsmax = max([np.abs(gc_absdiff.min()), np.abs(gc_absdiff.max())])
        [vmin, vmax] = [-diffabsmax, diffabsmax]
        ax2.coastlines()
        # assume the same grid for now in gchp vs gchp
        masked_csdata = np.ma.masked_where(np.abs(csgrid_dev['lon'] - 180) < 2, gc_absdiff)
        for i in range(6):
            plot2 = ax2.pcolormesh(csgrid_dev['lon_b'][i,:,:], csgrid_dev['lat_b'][i,:,:], 
                                   masked_csdata[i,:,:], cmap=WhGrYlRd,vmin=vmin, vmax=vmax)
        ax2.set_title('Difference\n(Dev - Ref)')   
        cb = plt.colorbar(plot2, ax=ax2, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.1 or (vmax-vmin) > 100:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)  
    
        # Subplot (1,1): Fractional Difference (restrict to +/-2)
        gc_fracdiff = (csdata_dev - csdata_ref) / csdata_ref
        if full_ratio_range: [vmin, vmax] = [None, None]
        else: [vmin, vmax] = [-2, 2]
        ax3.coastlines()
        # assume the same grid for now in gchp vs gchp
        masked_csdata = np.ma.masked_where(np.abs(csgrid_dev['lon'] - 180) < 2, gc_fracdiff)
        for i in range(6):
            plot3 = ax3.pcolormesh(csgrid_dev['lon_b'][i,:,:], csgrid_dev['lat_b'][i,:,:], 
                                   masked_csdata[i,:,:], cmap='RdBu_r',vmin=vmin, vmax=vmax)
        ax3.set_title('Fractional Difference\n(Dev - Ref)/Ref')   
        cb = plt.colorbar(plot3, ax=ax3, orientation='horizontal', pad=0.10)
        cb.set_clim(vmin=vmin, vmax=vmax)
        cb.set_label('unitless')     
            
        if savepdf:    
            pdf.savefig(figs)
            plt.close(figs)
            
    if savepdf: pdf.close()

### Function to plot and compare zonal means of GCHP

Note: Zonal mean is defined as the mean value across a constant latitudinal band. Calculating zonal mean from cubed sphere is not straight-forward. This function therefore regrids model outputs to a common lat-lon grid resolution prior to plotting. Many of the same features available for plotting a single level above are also available for this function.

In [12]:
def compare_zonal_mean(refdata, refstr, devdata, devstr, varlist=None, 
                       weightsdir='.', itime=0, llres_cmp='1x1.25', savepdf=False, 
                       pdfname='gchp_vs_gchp_map.pdf', match_cbar=True, full_ratio_range=False, 
                       normalize_by_area=False, area_ref=None, area_dev=None, flip_vert=False):

    # If no varlist is passed, plot all 3D variables in the dataset
    if varlist == None:
        [commonvars, commonvars2D, varlist] = compare_varnames(refdata, devdata)
        print('Plotting all 3D variables')
    n_var = len(varlist)
    
    # Get lat-lon grid
    llgrid_cmp = make_grid_LL(llres_cmp)

    # Get cubed sphere grid and regridder for first data set
    csres_ref = refdata['lon'].size
    [csgrid_ref, csgrid_list_ref] = make_grid_CS(csres_ref)
    cs_regridder_list_ref = make_regridder_C2L(csres_ref, llres_cmp, weightsdir=weightsdir, 
                                               reuse_weights=True)

    # Get cubed sphere grid and regridder for first data set
    csres_dev = devdata['lon'].size
    [csgrid_dev, csgrid_list_dev] = make_grid_CS(csres_dev)
    cs_regridder_list_dev = make_regridder_C2L(csres_dev, llres_cmp, weightsdir=weightsdir, 
                                               reuse_weights=True)
    
    # Universal plot setup
    xtick_positions = np.arange(-90,91,30)
    xticklabels = ['{}$\degree$'.format(x) for x in xtick_positions]
    ytick_positions = np.arange(0,61,20)
    yticklabels = [str(y) for y in ytick_positions]
    
    # Create pdf (if saving)
    if savepdf:
        print('\nCreating {} for {} variables'.format(pdfname, n_var))
        pdf = PdfPages(pdfname)

    # Loop over variables
    for ivar in range(n_var):
        if savepdf: print('{} '.format(ivar), end='')
        varname = varlist[ivar]
        
        # Do some checks: dimensions and units
        varndim_ref = refdata[varname].ndim
        varndim_dev = devdata[varname].ndim
        nlev = 72
        assert varndim_ref == varndim_dev, 'GCHP dimensions do not agree for {}!'.format(varname)
        units_ref = refdata[varname].units
        units_dev = devdata[varname].units
        assert units_ref == units_dev, 'GCHP units do not match for {}!'.format(varname)
        
        # Set plot extent
        extent=(-90,90,0,nlev)
        
        # if normalizing by area, adjust units to be per m2, and adjust title string
        units = units_ref
        varndim = varndim_ref
        subtitle_extra = ''
        
        # Slice the data
        ds_ref = refdata[varname].isel(time=itime)
        ds_dev = devdata[varname].isel(time=itime)

        # if normalizing by area, transform on the native grid and adjust units and subtitle string
        exclude_list = ['WetLossConvFrac','Prod_','Loss_']
        if normalize_by_area and not any(s in varname for s in exclude_list):
            ds_ref.values = ds_ref.values / area_ref.values[np.newaxis,:,:]
            ds_dev.values = ds_dev.values / area_dev.values[np.newaxis,:,:]
            units = '{} m-2'.format(units)
            subtitle_extra = ', Normalized by Area'
            
        # Regrid the slices
        if flip_vert: 
            ds_ref.data = ds_ref.data[::-1,:,:]
            ds_dev.data = ds_dev.data[::-1,:,:]
        csdata_ref = ds_ref.data.reshape(nlev,6,csres_ref,csres_ref).swapaxes(0,1)
        csdata_dev = ds_dev.data.reshape(nlev,6,csres_dev,csres_dev).swapaxes(0,1)
        lldata_ref = np.zeros([nlev, llgrid_cmp['lat'].size, llgrid_cmp['lon'].size])
        lldata_dev = np.zeros([nlev, llgrid_cmp['lat'].size, llgrid_cmp['lon'].size])
        for i in range(6):
            regridder_ref = cs_regridder_list_ref[i]
            lldata_ref += regridder_ref(csdata_ref[i])
            regridder_dev = cs_regridder_list_dev[i]
            lldata_dev += regridder_dev(csdata_dev[i])
        
        # Calculate zonal mean of the regridded data
        zm_ref = lldata_ref.mean(axis=2)
        zm_dev = lldata_dev.mean(axis=2)
            
        # Get min and max for colorbar limits
        [vmin_ref, vmax_ref] = [zm_ref.min(), zm_ref.max()]
        [vmin_dev, vmax_dev] = [zm_dev.min(), zm_dev.max()]
        vmin_cmn = np.min([vmin_ref, vmin_dev])
        vmax_cmn = np.max([vmax_ref, vmax_dev])
        if match_cbar: [vmin, vmax] = [vmin_cmn, vmax_cmn]
        
        # Create 2x2 figure
        figs, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=[12,12], 
                                                      subplot_kw={'projection': crs.PlateCarree()})
        # Give the page a title
        offset = 0.96
        fontsize=25
        figs.suptitle('{}, Zonal Mean'.format(varname), fontsize=fontsize, y=offset)

        # Subplot 0: Ref
        if not match_cbar: [vmin, vmax] = [vmin_ref, vmax_ref]
        plot0 = ax0.imshow(zm_ref, cmap=WhGrYlRd, extent=extent, vmin=vmin, vmax=vmax)
        ax0.set_title('{} (Ref){}\n{} regridded from {}'.format(refstr, subtitle_extra, 
                                                                llres_cmp, csres_ref))
        ax0.set_aspect('auto')
        ax0.set_xticks(xtick_positions)
        ax0.set_xticklabels(xticklabels)
        ax0.set_yticks(ytick_positions)
        ax0.set_yticklabels(yticklabels)
        cb = plt.colorbar(plot0, ax=ax0, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.001 or (vmax-vmin) > 1000:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)
        
        # Subplot 1: Dev
        if not match_cbar: [vmin, vmax] = [vmin_dev, vmax_dev]
        plot1 = ax1.imshow(zm_dev, cmap=WhGrYlRd, extent=extent, vmin=vmin, vmax=vmax)
        ax1.set_title('{} (Dev){}\n{} regridded from {}'.format(devstr, subtitle_extra, 
                                                                llres_cmp, csres_dev))
        ax1.set_aspect('auto')
        ax1.set_xticks(xtick_positions)
        ax1.set_xticklabels(xticklabels)
        ax1.set_yticks(ytick_positions)
        ax1.set_yticklabels(yticklabels)
        cb = plt.colorbar(plot1, ax=ax1, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.001 or (vmax-vmin) > 1000:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)
            
        # Subplot 2: Difference
        zm_absdiff = zm_dev - zm_ref
        diffabsmax = max([np.abs(zm_absdiff.min()), np.abs(zm_absdiff.max())])
        [vmin, vmax] = [-diffabsmax, diffabsmax]
        plot2 = ax2.imshow(zm_absdiff, cmap='RdBu_r', extent=extent, vmin=vmin, vmax=vmax)
        ax2.set_title('Difference\n(Dev - Ref)')
        ax2.set_aspect('auto')
        ax2.set_xticks(xtick_positions)
        ax2.set_xticklabels(xticklabels)
        ax2.set_yticks(ytick_positions)
        ax2.set_yticklabels(yticklabels)
        cb = plt.colorbar(plot2, ax=ax2, orientation='horizontal', pad=0.10)
        if (vmax-vmin) < 0.001 or (vmax-vmin) > 1000:
            cb.locator = ticker.MaxNLocator(nbins=4)
            cb.update_ticks()
        cb.set_label(units)
        
        # Subplot 3: Fractional Difference (restrict to +/-2)
        zm_fracdiff = (zm_dev - zm_ref) / zm_ref
        if full_ratio_range: [vmin, vmax] = [None, None]
        else: [vmin, vmax] = [-2, 2]
        plot3 = ax3.imshow(zm_fracdiff, vmin=vmin, vmax=vmax, cmap='RdBu_r', extent=extent)
        ax3.set_title('Fractional Difference\n(Dev - Ref)/Ref')
        ax3.set_aspect('auto')
        ax3.set_xticks(xtick_positions)
        ax3.set_xticklabels(xticklabels)
        ax3.set_yticks(ytick_positions)
        ax3.set_yticklabels(yticklabels)
        cb = plt.colorbar(plot3, ax=ax3, orientation='horizontal', pad=0.10)
        cb.set_clim(vmin=vmin, vmax=vmax)
        cb.set_label('unitless')      
            
        if savepdf:    
            pdf.savefig(figs)
            plt.close(figs)
            
    if savepdf: pdf.close()

## Define directories and global information

In [13]:
# Shared high-level directory
testdir = '/n/home08/elundgren/GC/testruns/12.0.3'

# Ref and dev run output directories
refdir = os.path.join(testdir,'gchp_standard_ref/OutputDir')
devdir = os.path.join(testdir,'gchp_standard_dev/OutputDir')

# Ref and dev strings (for use in plots and messages)
refstr='12.0.2'
devstr='12.0.3'

# Set directory to store plots
plotsdir = os.path.join(testdir,'plots')

# Set directory to store regridding weights
weightsdir = '/n/home08/elundgren/GC/python/regrid_weights'

# Define cubed sphere resolutions (used for restart file comparison)
csres_ref = 48
csres_dev = 48

# Check that paths exist
check_paths(refdir, devdir)

Path 1 exists: /n/home08/elundgren/GC/testruns/12.0.3/gchp_standard_ref/OutputDir
Path 2 exists: /n/home08/elundgren/GC/testruns/12.0.3/gchp_standard_dev/OutputDir


### Print Ref netcdf filenames

In [14]:
reffiles = [k for k in os.listdir(refdir) if '.nc' in k]
for k in reffiles:
    print(k)

GCHP.Emissions.20160716_1200z.nc4
GCHP.LevelEdgeDiags.20160716_1200z.nc4
GCHP.Aerosols.20160716_1200z.nc4
GCHP.JValues.20160716_1200z.nc4
GCHP.ConcAfterChem.20160716_1200z.nc4
GCHP.WetLossConv.20160716_1200z.nc4
GCHP.StateMet_inst.20160801_0000z.nc4
GCHP.DryDep.20160716_1200z.nc4
GCHP.ProdLoss.20160716_1200z.nc4
GCHP.SpeciesConc_inst.20160801_0000z.nc4
GCHP.WetLossLS.20160716_1200z.nc4
GCHP.SpeciesConc_avg.20160716_1200z.nc4
GCHP.CloudConvFlux.20160716_1200z.nc4
GCHP.StateMet_avg.20160716_1200z.nc4
GCHP.StateChm.20160716_1200z.nc4
GCHP.AerosolMass.20160716_1200z.nc4


### Print Dev netcdf filenames

In [15]:
devfiles = [k for k in os.listdir(devdir) if '.nc' in k]
for k in devfiles:
    print(k)

GCHP.Emissions.20160716_1200z.nc4
GCHP.LevelEdgeDiags.20160716_1200z.nc4
GCHP.Aerosols.20160716_1200z.nc4
GCHP.JValues.20160716_1200z.nc4
GCHP.ConcAfterChem.20160716_1200z.nc4
GCHP.WetLossConv.20160716_1200z.nc4
GCHP.StateMet_inst.20160801_0000z.nc4
GCHP.DryDep.20160716_1200z.nc4
GCHP.ProdLoss.20160716_1200z.nc4
GCHP.SpeciesConc_inst.20160801_0000z.nc4
GCHP.WetLossLS.20160716_1200z.nc4
GCHP.SpeciesConc_avg.20160716_1200z.nc4
GCHP.CloudConvFlux.20160716_1200z.nc4
GCHP.StateMet_avg.20160716_1200z.nc4
GCHP.StateChm.20160716_1200z.nc4
GCHP.AerosolMass.20160716_1200z.nc4


## Create Regridding Weights (executing this section is not required)

The xESMF Regridder object can create weights for regridding from one grid resolution to another, or it can be provided weights that are pre-generated to save time. Generating weights therefore only needs to be done once for each combination of input and output grid type and resolution. This can be done by calling the make_regridder functions and passing reuse_weights=False. Once the weights exist, all subsequent calls to make a regridder can be passed reuse_weights=True along with the location of the pre-existing weights.

In [16]:
# This is here only as an example. Plotting will create the weights on-the-fly if they do not yet exist.
#regridder_C2L_list = make_regridder_C2L(48, '1x1.25', weightsdir=weightsdir, reuse_weights=False)

## Compare Restart Files

In [17]:
reffile_rst = os.path.join(refdir,'..','initial_GEOSChem_rst.c{}_standard.nc'.format(csres_ref))
devfile_rst = os.path.join(devdir,'..','initial_GEOSChem_rst.c{}_standard.nc'.format(csres_dev))
check_paths(reffile_rst, devfile_rst)

Path 1 exists: /n/home08/elundgren/GC/testruns/12.0.3/gchp_standard_ref/OutputDir/../initial_GEOSChem_rst.c48_standard.nc
Path 2 exists: /n/home08/elundgren/GC/testruns/12.0.3/gchp_standard_dev/OutputDir/../initial_GEOSChem_rst.c48_standard.nc


In [18]:
refdata_rst = xr.open_dataset(reffile_rst)
devdata_rst = xr.open_dataset(devfile_rst)
[commonvars, commonvars2D, commonvars3D] = compare_varnames(refdata_rst, refstr, devdata_rst, devstr)

277 common variables (0 are 2-dim and 277 are 3-dim)


In [19]:
get_stats(refdata_rst, refstr, devdata_rst, devstr, 'SPC_CFC11')

Data units: mol mol-1
Array sizes:
    12.0.2:  (1, 72, 288, 48)
    12.0.3:  (1, 72, 288, 48)
Global stats:
  Mean:
    12.0.2:  1.1734857530143472e-10
    12.0.3:  1.1734857530143472e-10
  Min:
    12.0.2:  0.0
    12.0.3:  0.0
  Max:
    12.0.2:  2.2552185119373291e-10
    12.0.3:  2.2552185119373291e-10
  Sum:
    12.0.2:  0.00011680032912408933
    12.0.3:  0.00011680032912408933


### Comparing restarts requires flipping GCHP in the vertical. Do this by passing flip_vert=True

In [20]:
#compare_zonal_mean(refdata_rst, refstr, devdata_rst, devstr,
#                   varlist=['SPC_O3'], weightsdir=weightsdir, flip_vert=True )

In [21]:
#compare_single_level(refdata_rst, refstr, devdata_rst, devstr,
#                     varlist=['SPC_O3'], ilev=0, weightsdir=weightsdir, flip_vert=True )

## Store Grid Areas for Normalizing by Area

Eventually this will be replaced by retrieving areas from xESMF.

In [22]:
day = '20160716'
time = '1200'
[refdata, devdata] = get_collection_data(refdir, devdir, 'StateMet_avg', day, time)
varname = 'Met_AREAM2'
ref_area = refdata[varname].isel(lev=0,time=0)
dev_area = devdata[varname].isel(lev=0,time=0)

Path 1 exists: /n/home08/elundgren/GC/testruns/12.0.3/gchp_standard_ref/OutputDir/GCHP.StateMet_avg.20160716_1200z.nc4
Path 2 exists: /n/home08/elundgren/GC/testruns/12.0.3/gchp_standard_dev/OutputDir/GCHP.StateMet_avg.20160716_1200z.nc4


## Inspect time-averaged species concentration diagnostic

In [23]:
day = '20160716'
time = '1200'
collection = 'SpeciesConc_avg'
[refdata, devdata] = get_collection_data(refdir, devdir, collection, day, time)

Path 1 exists: /n/home08/elundgren/GC/testruns/12.0.3/gchp_standard_ref/OutputDir/GCHP.SpeciesConc_avg.20160716_1200z.nc4
Path 2 exists: /n/home08/elundgren/GC/testruns/12.0.3/gchp_standard_dev/OutputDir/GCHP.SpeciesConc_avg.20160716_1200z.nc4


In [24]:
[commonvars, commonvars2D, commonvars3D] = compare_varnames(refdata, refstr, devdata, devstr)

161 common variables (0 are 2-dim and 161 are 3-dim)


In [25]:
CFCs=[k for k in commonvars if 'CFC' in k]
print(CFCs)

['SpeciesConc_CFC11', 'SpeciesConc_CFC113', 'SpeciesConc_CFC114', 'SpeciesConc_CFC115', 'SpeciesConc_CFC12', 'SpeciesConc_HCFC123', 'SpeciesConc_HCFC141b', 'SpeciesConc_HCFC142b', 'SpeciesConc_HCFC22']


In [26]:
varname='SpeciesConc_CFC11'
get_stats(refdata, refstr, devdata, devstr, varname)

Data units: mol mol-1 dry
Array sizes:
    12.0.2:  (1, 72, 288, 48)
    12.0.3:  (1, 72, 288, 48)
Global stats:
  Mean:
    12.0.2:  5.978070816148318e-11
    12.0.3:  1.149659673016501e-10
  Min:
    12.0.2:  0.0
    12.0.3:  0.0
  Max:
    12.0.2:  2.0787468968386236e-10
    12.0.3:  2.1939146332972115e-10
  Sum:
    12.0.2:  5.9501409850781783e-05
    12.0.3:  0.00011442884715506807


In [27]:
#compare_single_level(refdata, refstr, devdata, devstr, varlist=[varname], weightsdir=weightsdir)

In [28]:
#compare_zonal_mean(refdata, refstr, devdata, devstr, varlist=[varname], weightsdir=weightsdir )

## Create PDF of plots for list of species

In [29]:
# Define subset of variables in files
#varlist = commonvars3D
varlist = [varname]
#varlist = [k for k in commonvars3D if 'CFC' in k]

In [30]:
# Surface
#pdfname = os.path.join(plotsdir,'{}_Surface.pdf'.format(ref_collection))
pdfname = os.path.join(plotsdir,'{}_Surface.pdf'.format(varname))
#pdfname = os.path.join(plotsdir,'{}_Surface.pdf'.format('CFCs'))
compare_single_level(refdata, refstr, devdata, devstr, varlist=varlist, ilev=0,
                     weightsdir=weightsdir, savepdf=True, pdfname=pdfname )


Creating /n/home08/elundgren/GC/testruns/12.0.3/plots/SpeciesConc_CFC11_Surface.pdf for 1 variables
0 

In [31]:
# Zonal mean
#pdfname = os.path.join(plotsdir,'{}_ZonalMean.pdf'.format(ref_collection))
pdfname = os.path.join(plotsdir,'{}_ZonalMean.pdf'.format(varname))
#pdfname = os.path.join(plotsdir,'{}_ZonalMean.pdf'.format('CFCs'))
compare_zonal_mean(refdata, refstr, devdata, devstr, varlist=varlist, 
                   weightsdir=weightsdir, savepdf=True, pdfname=pdfname )


Creating /n/home08/elundgren/GC/testruns/12.0.3/plots/SpeciesConc_CFC11_ZonalMean.pdf for 1 variables
0 

## Other diagnostic collections

In [32]:
# Inspect the available files again, but only show those that match
commonfiles = [k for k in reffiles if k in devfiles]
commonfiles

['GCHP.Emissions.20160716_1200z.nc4',
 'GCHP.LevelEdgeDiags.20160716_1200z.nc4',
 'GCHP.Aerosols.20160716_1200z.nc4',
 'GCHP.JValues.20160716_1200z.nc4',
 'GCHP.ConcAfterChem.20160716_1200z.nc4',
 'GCHP.WetLossConv.20160716_1200z.nc4',
 'GCHP.StateMet_inst.20160801_0000z.nc4',
 'GCHP.DryDep.20160716_1200z.nc4',
 'GCHP.ProdLoss.20160716_1200z.nc4',
 'GCHP.SpeciesConc_inst.20160801_0000z.nc4',
 'GCHP.WetLossLS.20160716_1200z.nc4',
 'GCHP.SpeciesConc_avg.20160716_1200z.nc4',
 'GCHP.CloudConvFlux.20160716_1200z.nc4',
 'GCHP.StateMet_avg.20160716_1200z.nc4',
 'GCHP.StateChm.20160716_1200z.nc4',
 'GCHP.AerosolMass.20160716_1200z.nc4']

In [33]:
# Template for inspecting any collection:

# To specify file:
#day = {edit}
#time = {edit}
#collection = {edit}

# For retrieving the data:
#[refdata, devdata] = get_collection_data(refdir, devdir, collection, day, time)

# For inspecting interactively:
#refdata
#devdata
#[commonvars, commonvars2D, commonvars3D] = compare_varnames(refdata, refstr, devdata, devstr)
# varname = {edit}
#get_stats(refdata, refstr, devdata, devstr, varname)
# varlist = {edit}
#compare_single_level(refdata, refstr, devdata, devstr, varlist=[varname], weightsdir=weightsdir)
#compare_zonal_mean(refdata, refstr, devdata, devstr, varlist=[varname], weightsdir=weightsdir )

# For creating PDFs:
#prefix = {edit}
#pdfname = os.path.join(plotsdir,'{}_Surface.pdf'.format(prefix))
#compare_single_level(refdata, refstr, devdata, devstr, varlist=varlist, ilev=0,
#                     weightsdir=weightsdir, savepdf=True, pdfname=pdfname )
#pdfname = os.path.join(plotsdir,'{}_ZonalMean.pdf'.format(prefix))
#compare_zonal_mean(refdata, refstr, devdata, devstr, varlist=varlist, 
#                   weightsdir=weightsdir, savepdf=True, pdfname=pdfname )

# For other function options use shift-tab while hovering over the function name in a cell that calls it